# Workflow for deepTools ChIP-Seq Heatmap Generation

The following notebook shows the optimized workflow for creating heatmaps starting from wig.gz files. Will use a combination of UCSC and deepTools utilities.

## Tools download and setup

In [ ]:
#Download the python program wigToBigWig from the UCSC's utilties library.

!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/wigToBigWig
!chmod +x ./wigToBigWig

#If you haven't done so yet, download deepTools using the following line

!pip install deeptools==3.5.4.post1

#When generating BigWig files, will need a file containing chromosome sizes

!wget https://hgdownload.cse.ucsc.edu/goldenpath/mm10/bigZips/mm10.chrom.sizes
chrom_size = "mm10.chrom.sizes"

#For sorting bed files for filtering, downlaod bedops

!conda install bioconda::bedops

## Preparing Wig Files for deepTools Heatmaps Creation

### Wig Files List Generator

To automate the process, we will generate a list that will find the right file according to a sample_key.txt file and implement it accordingly. In our dataset, the wig files were labeled using an index code that would only be deciphered using the sample_key to figure out histone modification mark and their experimental group.

In [1]:
import pandas as pd

# sample_key.txt (file with name conversion and other info) has been changed to a csv file through excel

key = pd.read_csv("combined_sample_key.csv")
key.head()

,project,assays,species,mark,sample name,index
0,Underhill,Chipseq,mouse,H3K4me1,SSM2_09June23_D_H3K4me1,PX3242_TGACCA-CAGTACAG
1,Underhill,Chipseq,mouse,H3K4me3,SSM2_09June23_D_H3K4me3,PX3242_CTTGTA-CAGTACAG
2,Underhill,Chipseq,mouse,H3K9me3,SSM2_09June23_D_H3K9me3,PX3242_AAGCGA-CAGTACAG
3,Underhill,Chipseq,mouse,H3K27me3,SSM2_09June23_D_H3K27me3,PX3242_ACTCTC-CAGTACAG
4,Underhill,Chipseq,mouse,H3K36me3,SSM2_09June23_D_H3K36me3,PX3242_ATACGG-CAGTACAG


In [2]:
# Filter the dataframe for only rows that correspond to ChIP-Seq data (marked as Chipseq in the assays column)

ChIP_Seq = key[key["assays"] == "Chipseq"]
ChIP_Seq

,project,assays,species,mark,sample name,index
0,Underhill,Chipseq,mouse,H3K4me1,SSM2_09June23_D_H3K4me1,PX3242_TGACCA-CAGTACAG
1,Underhill,Chipseq,mouse,H3K4me3,SSM2_09June23_D_H3K4me3,PX3242_CTTGTA-CAGTACAG
2,Underhill,Chipseq,mouse,H3K9me3,SSM2_09June23_D_H3K9me3,PX3242_AAGCGA-CAGTACAG
3,Underhill,Chipseq,mouse,H3K27me3,SSM2_09June23_D_H3K27me3,PX3242_ACTCTC-CAGTACAG
4,Underhill,Chipseq,mouse,H3K36me3,SSM2_09June23_D_H3K36me3,PX3242_ATACGG-CAGTACAG
5,Underhill,Chipseq,mouse,H3K27ac,SSM2_09June23_D_H3K27ac,PX3242_CACGAT-CAGTACAG
6,Underhill,Chipseq,mouse,H3K36me2,SSM2_09June23_D_H3K36me2,PX3242_GAGTGG-CAGTACAG
7,Underhill,Chipseq,mouse,Ubiquityl,SSM2_09June23_D_Ubiquityl,PX3242_CTGCTG-CAGTACAG
8,Underhill,Chipseq,mouse,Input,SSM2_09June23_D_Input,PX3242_TTCTCC-GTAGTACT
9,Underhill,Chipseq,mouse,H3K4me1,CTL_Tongue_10Aug23_G_H3K4me1,PX3242_ACAGTG-GTAGTACT


### Converting Wig Files to BigWig

BigWig (bw) files are necessary due to deepTools using that format as their input, but also they come in handy when utilzing IGV program. When trying to open wig files in IGV, they will reccomend converting it to TDF using their built-in tools, but that can take hours to process. Instead, converting to bw format takes only a couple of minutes. We will use the program wigToBigWig, however we must first unzip the files if that are compressed as .gz with a linux function called gzip.

In [ ]:
# Using gzip to unzip the .gz compress wig files.
# To automate this process, run a loop to gather all the names of the files from the samples_key file, and then feed those names through another loop that will unzip each one sequentially.

compressed_list = []

for i in range(len(ChIP_Seq["index"])):
    row = ChIP_Seq.iloc[i]
    if "E12_5" in row["index"]:
        compressed_list.append(row["index"] + ".wig.gz")
    else:
        compressed_list.append(row["index"] + ".markdup.q5.F3332.PET.wig.gz")

for name in compressed_list:
    !gzip -dv -k {name}

In [ ]:
#Generating bw files from the wig files using UCSC's wigToBigWig tool

chrom_size = "mm10.chrom.sizes"

for name in ChIP_Seq['index']:
    if "E12_5" in name:
        !./wigToBigWig {name + ".wig"} {chrom_size} {name + ".bw"} -clip
    else:
        !./wigToBigWig {name + ".markdup.q5.F3332.PET.wig"} {chrom_size} {name + ".markdup.q5.F3332.PET.bw"} -clip

### BigWig Files List Generator

Create a list that will have the E12_5, CTL, and SSM2 bw files in a row for each corresponding histone type.

In [4]:
# Find all the unique markers used in the ChIP-Seq data to form pairs of CTL vs SSM2 later (found in the mark column)

marks = set(ChIP_Seq["mark"])
marks

{'H3K27ac',
 'H3K27me3',
 'H3K36me2',
 'H3K36me3',
 'H3K4me1',
 'H3K4me3',
 'H3K9me3',
 'Input',
 'Ubiquityl'}

In [5]:
# Make empty lists that will hold the string names of the markers, CTL index names, and the SSM2 index names

mark_list = []
E12_5_list = []
CTL_list = []
SSM2_list = []

In [6]:
# The following is a for loop that will ultimately scan, pull, and insert the index names of CTL and SSM2 bigwig files pairs

for mark in marks:  # A for loop that will go through each unique marker name from the dataframe
    E12_5 = ChIP_Seq[(ChIP_Seq["mark"] == mark) & (ChIP_Seq["index"].str.contains("E12_5"))]
    E12_5_index = E12_5["index"].values[0]
    CTL = ChIP_Seq[(ChIP_Seq["mark"] == mark) & (ChIP_Seq["sample name"].str.contains("CTL"))] # Collect row that contains the right marker type and has the substring "CTL" in the sample name column
    CTL_index = CTL["index"].values[0] # Copy just the string of the index name from the CTL row. Have to use .values[0] or else you get a string that contains the row number and other useless data that will interfere with getting the right bigwig file name.
    SSM2 = ChIP_Seq[(ChIP_Seq["mark"] == mark) & (ChIP_Seq["sample name"].str.contains("SSM2"))] # Repeat of the CTL portion, but instead we find the row with the SSM2
    SSM2_index = SSM2["index"].values[0]
    E12_5_bw = E12_5_index + ".bw"
    CTL_bw = CTL_index + ".markdup.q5.F3332.PET.bw" # Add the remaining text from the bigwig file in order to call it properly
    SSM2_bw = SSM2_index + ".markdup.q5.F3332.PET.bw"
    mark_list.append(mark) # Add the current marker name to the mark_list list
    E12_5_list.append(E12_5_bw)
    CTL_list.append(CTL_bw) # Add the full name of the CTL bigwig file to the CTL_list list
    SSM2_list.append(SSM2_bw) # Add the full name of the SSM2 bigwig file to the SSM2_list list
    

In [7]:
# Make a dataframe that will house columns for marker type, and CTL & SSM2 bigwig names.
import pandas as pd

bw_table = pd.DataFrame({'mark':mark_list, 'E12_5':E12_5_list, 'CTL':CTL_list, 'SSM2':SSM2_list})
bw_table

,mark,E12_5,CTL,SSM2
0,H3K36me2,E12_5FL.H3K36me2.q5.F1028.PET.bw,PX3242_CCGCAA-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_GAGTGG-CAGTACAG.markdup.q5.F3332.PET.bw
1,H3K9me3,E12_5FL.H3K9me3.q5.F1028.PET.bw,PX3242_AAGGAC-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_AAGCGA-CAGTACAG.markdup.q5.F3332.PET.bw
2,Input,E12_5FL.Input.q5.F1028.PET.bw,PX3242_GTGAAA-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_TTCTCC-GTAGTACT.markdup.q5.F3332.PET.bw
3,Ubiquityl,E12_5FL.H2AK119ub.q5.F1028.PET.bw,PX3242_GAAACC-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_CTGCTG-CAGTACAG.markdup.q5.F3332.PET.bw
4,H3K27ac,E12_5FL.H3K27ac.q5.F1028.PET.bw,PX3242_CACTCA-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_CACGAT-CAGTACAG.markdup.q5.F3332.PET.bw
5,H3K4me3,E12_5FL.H3K4me3.q5.F1028.PET.bw,PX3242_AAACAT-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_CTTGTA-CAGTACAG.markdup.q5.F3332.PET.bw
6,H3K27me3,E12_5FL.H3K27me3.q5.F1028.PET.bw,PX3242_ACTGAT-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_ACTCTC-CAGTACAG.markdup.q5.F3332.PET.bw
7,H3K36me3,E12_5FL.H3K36me3.q5.F1028.PET.bw,PX3242_ATCCTA-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_ATACGG-CAGTACAG.markdup.q5.F3332.PET.bw
8,H3K4me1,E12_5FL.H3K4me1.q5.F1028.PET.bw,PX3242_ACAGTG-GTAGTACT.markdup.q5.F3332.PET.bw,PX3242_TGACCA-CAGTACAG.markdup.q5.F3332.PET.bw


### Log2 normalizing BigWig files

For some of the heatmaps, we will use log2 normalization, which requires using the deeptools tool bigwigCompare. Two different bw files are needed, treatment bw file and a input bw file.

In [ ]:
#The following code is used if log2 normalization is needed for the heatmap.
#Isolate the input row which will be used as the reference bw file during log2 normalization.

input = bw_table[bw_table.eq("Input").any(axis=1)]

#Make a for loop that will run through the bw_table list and utilize deeptools' bigWigCompare to normalize every bw file to its input file (CTL or SSM2 respectively) with bin sizes of 25.
#Takes ~40 mins to run log2 normalization from ~1 GB bw files. Normalized bw files are around 600 MB. Number of processors is set to 25 - change accordingly.

for i in range(len(bw_table['mark'])):
    if bw_table.loc[i, 'mark'] != "Input":
        !bigwigCompare -b1 {bw_table.loc[i, 'E12_5']} -b2 {input['E12_5'].values[0]} -bs 25 --operation log2 -o {bw_table.loc[i,'mark'] + "_E12_5_log2.bw"} -p 25
        !bigwigCompare -b1 {bw_table.loc[i, 'CTL']} -b2 {input['CTL'].values[0]} -bs 25 --operation log2 -o {bw_table.loc[i,'mark'] + "_CTL_log2.bw"} -p 25
        !bigwigCompare -b1 {bw_table.loc[i, 'SSM2']} -b2 {input['SSM2'].values[0]} -bs 25 --operation log2 -o {bw_table.loc[i,'mark'] + "_SSM2_log2.bw"} -p 25

## Generating Heatmaps

Before creating a heatmap, we must first generate an intermediate matrix file that will house score values of different genes upstream and downstream of the region of interest based on the bw file and a reference region bed file (gene list).

We used a variety of bed file that either included all the genes found in the mm10 genomes or a curated gene list.
1. For the former, it can be downloaded from [here](https://sourceforge.net/projects/rseqc/files/BED/Mouse_Mus_musculus/mm10_UCSC_knownGene.bed.gz/download).
2. For the latter, it can be generated using [UCSC's table browser website](https://genome.ucsc.edu/cgi-bin/hgTables?hgsid=2125137146_54BnotAR6qPQ2C8qFACtCzdewo7Y&clade=mammal&org=Mouse&db=mm10&hgta_group=genes&hgta_track=refSeqComposite&hgta_table=refGene&hgta_regionType=genome&position=chr12%3A56%2C694%2C976-56%2C714%2C605&hgta_outputType=primaryTable&hgta_outFileName=). From trial runs, the most polished bed file was generated using the NCBI refseq track and UCSC refseq table. For identifiers, input a txt file that contains a list of genes. Export as a bed file with no headers. Due to the gene data set, they will occassionally have multiple versions of the same gene list, creating repetitions of one single gene in the bed file. This will mess up the heatmap due to the pressence of large and broad streaks. To fix this, open the bed file using a txt file editor (for example: Notepad on Windows) and delete rows that have similar chromosome, start, and end values. To make things easier, should sort the bed file using bedops' sort_bed program.

In [ ]:
import pyranges as pr
import os

#Prepare bed files containing the start and end positions of genes using the USCS table browser tool. Specifically, utilizing the NCBI dataset and UCSC refseq table.
#Checked bed file using IGV.

cluster_1 = "mm10_cluster1_genelist_NCBI_UCSC.bed"
cluster_2 = "mm10_cluster2_genelist_NCBI_UCSC.bed"
cluster_3 = "mm10_cluster3_genelist_NCBI_UCSC.bed"
Hox_genes = "mm10_HoxGenes_genelist_NCBI_UCSC.bed"


In [ ]:
# Sorting gene list bed file to make filtering easier

bed_list = [cluster_1, cluster_2, cluster_3, Hox_genes]

for bed in bed_list:
    !sort-bed {bed} > {"sorted_" + bed}

In [ ]:
# Removing duplicates of genes from the bed file based on exact same chromosome, start, and end.

for bed in bed_list:
    !sort -k1,1 -k2,2n -u {"sorted_" + bed} -o {"filtered_sorted_" + bed}
    !sort -k1,1 -k3,3n -u {"filtered_sorted_" + bed} -o {"filtered_sorted_" + bed}

# IMPORTANT
# There will still be duplicates of genes that remain due to small differences in start and end values. Will have to go through it manually and remove them. 

### Heatmaps around TSS, using absolute values, and all mm10 genes

The following will generate pairs of heatmaps for each histone modification with the CTL group on the left and SSM2 group on the right

In [ ]:
# Using the bw_table, we will now go through each row/marker type and input the CTL and SSM2 bigwig files into the deeptools computeMatrix program
# This will create a matrix from both CTL and SSM2 bigwigs that can later be used to create heatmaps

for i in range(len(bw_table['mark'])):
    !computeMatrix reference-point -S {bw_table.loc[i, 'CTL']} {bw_table.loc[i, 'SSM2']} -R mm10_UCSC_knownGene.bed -bs 25 -b 5000 -a 5000  -out {bw_table.loc[i, 'mark'] + "_CTL_SSM2_referencePoint.matrix"} --missingDataAsZero --skipZeros --samplesLabel {bw_table.loc[i, 'mark'] + "_CTL"} {bw_table.loc[i, 'mark'] + "_SSM2"} -p 20

# Similar to the code above for running computeMatrix, we will use the bw_table to run deeptool's plotHeatmap
# This will make heatmap pairs of CTL and SSM2

for i in range(len(bw_table['mark'])):
    !plotHeatmap -m {bw_table.loc[i, 'mark'] + "_CTL_SSM2_referencePoint.matrix"} --colorMap RdBu --heatmapHeight 25 --heatmapWidth 5 -out {bw_table.loc[i, 'mark'] + "_CTL_SSM2_referencePoint.svg"} --sortUsing max --regionsLabel genes --legendLocation none

### Heatmaps around TSS, using log2 normalized values, and all mm10 genes

The following will generate pairs of heatmaps for each histone modification with the CTL group on the left and SSM2 group on the right

In [ ]:
# Using the bw_table, we will now go through each row/marker type and input the CTL and SSM2 bigwig files into the deeptools computeMatrix program
# This will create a matrix from both CTL and SSM2 bigwigs that can later be used to create heatmaps

for i in range(len(bw_table['mark'])):
    !computeMatrix reference-point -S {bw_table.loc[i, 'mark'] + "_CTL_log2.bw"} {bw_table.loc[i, 'mark'] + "_SSM2_log2.bw"} -R mm10_UCSC_knownGene.bed -bs 25 -b 5000 -a 5000  -out {bw_table.loc[i, 'mark'] + "_CTL_SSM2_log2_reference_point.matrix"} --missingDataAsZero --skipZeros --samplesLabel {bw_table.loc[i, 'mark'] + "_CTL"} {bw_table.loc[i, 'mark'] + "_SSM2"} -p 15

# Similar to the code above for running computeMatrix, we will use the bw_table to run deeptool's plotHeatmap
# This will make heatmap pairs of CTL and SSM2

for i in range(len(bw_table['mark'])):
    !plotHeatmap -m {bw_table.loc[i, 'mark'] + "_CTL_SSM2_log2_referencePoint.matrix"} --colorMap RdBu --heatmapHeight 25 --heatmapWidth 5 -out {bw_table.loc[i, 'mark'] + "_CTL_SSM2_log2_referencePoint.svg"} --sortUsing max --regionsLabel genes --legendLocation none

### Combined heatmaps around TSS, using absolute values, and curated gene lists

All the bw files and different histone modifications were implemented and processed together. 2 different heatmaps will ultimately be generated, with one utilizing 3 different bed files (cluster 1, cluster 2, and cluster 3), and the other only used a list of Hox genes.

In [ ]:
#For the sake of easier name calling of bw files for combined heatmaps, we will rename them with more obvious names.
#To do this, we will run a for loop on the bw_table and use the os.rename() function.

for i in range(len(bw_table['mark'])):
    for n in range(len(bw_table.columns)-1):
        old_name = bw_table.loc[i, bw_table.columns[n+1]]
        new_name = bw_table.loc[i, bw_table.columns[0]] + "_" + bw_table.columns[n+1] + ".bw"
        print("Old name = " + old_name + " New name = " +  new_name)
        os.rename(old_name, new_name)

In [ ]:
#Rather than writing out 24 different file names and 24 different labels, we will automate the process.

histone_order = ['H3K4me3', 'H3K27ac', 'H3K27me3', 'Ubiquityl', 'H3K4me1', 'H3K9me3', 'H3K36me2', 'H3K36me3'] #Specify the order of histone modifications.
bw_list = ''
histone_list = ''
for histone in histone_order:
    bw_list += (histone + "_E12_5.bw ") #Specifiy the order of bw files per histone modification type (for this example: E12.5, CTL, then SSM2).
    bw_list += (histone + "_CTL.bw ")
    bw_list += (histone + "_SSM2.bw ")
    histone_list += (histone + "_E12_5 ") #The order of heatmap labels corresponding to the order of bw files.
    histone_list += (histone + "_CTL ")
    histone_list += (histone + "_SSM2 ")

In [ ]:
#Compute a heatmap matrix that combines all 16 different bw files in a specific order and based around select regions using bed files.
#The following will utilize non-normalized bw files.
#Two separate heatmaps will be ultimately generated, one with 3 different clusters using gene lists, and one that uses Hox genes.
#Will generate matrices that calculate scores 1 kb upstream and downstream of the transciption start site (TSS).

!computeMatrix reference-point \
    -S {bw_list} \
    -R filtered_sorted_mm10_cluster1_genelist_NCBI_UCSC.bed filtered_sorted_mm10_cluster2_genelist_NCBI_UCSC.bed filtered_sorted_mm10_cluster3_genelist_NCBI_UCSC.bed \
    -bs 25 -b 5000 -a 5000  -out E12_5CTL_SSM2_TSS_5kb_clusters.matrix --missingDataAsZero --skipZeros \
    --samplesLabel {histone_list} \
    -p 25 --verbose

!computeMatrix reference-point \
    -S {bw_list} \
    -R filtered_sorted_mm10_HoxGenes_genelist_NCBI_UCSC.bed \
    -bs 25 -b 5000 -a 5000  -out E12_5_CTL_SSM2_TSS_5kb_HoxGenes.matrix --missingDataAsZero --skipZeros \
    --samplesLabel {histone_list} \
    -p 25 --verbose

In [ ]:
#Generate heatmaps in svg format for both types.

!plotHeatmap -m E12_5_CTL_SSM2_TSS_5kb_clusters.matrix --colorMap RdBu --heatmapHeight 25 --heatmapWidth 5 \
    -out E12_5_CTL_SSM2_TSS_5kb_clusters.svg --sortUsing max --legendLocation none \
    --regionsLabel "Cluster 1" "Cluster 2" "Cluster 3" --whatToShow "heatmap and colorbar" --outFileSortedRegions "E12_5_CTL_SSM2_TSS_5kb_clusters.bed"

plotHeatmap -m E12_5_CTL_SSM2_TSS_5kb_HoxGenes.matrix --colorMap RdBu --heatmapHeight 25 --heatmapWidth 5 \
    -out E12_5_CTL_SSM2_TSS_5kb_HoxGenes.svg --sortUsing max --legendLocation none \
    --regionsLabel "Hox Genes" --whatToShow "heatmap and colorbar" --outFileSortedRegions "E12_5_CTL_SSM2_TSS_5kb_HoxGenes.bed"

### Combined heatmaps around TSS, using log2 normailzed values, and curated gene lists

All the bw files and different histone modifications were implemented and processed together. 2 different heatmaps will ultimately be generated, with one utilizing 3 different bed files (cluster 1, cluster 2, and cluster 3), and the other only used a list of Hox genes.

In [ ]:
#Rather than writing out 24 different file names and 24 different labels, we will automate the process.

histone_order = ['H3K4me3', 'H3K27ac', 'H3K27me3', 'Ubiquityl', 'H3K4me1', 'H3K9me3', 'H3K36me2', 'H3K36me3'] #Specify the order of histone modifications.
bw_log2_list = ''
histone_list = ''
for histone in histone_order:
    bw_log2_list += (histone + "_E12_5_log2.bw ") #Specifiy the order of bw files per histone modification type (for this example: E12.5, CTL, then SSM2).
    bw_log2_list += (histone + "_CTL_log2.bw ")
    bw_log2_list += (histone + "_SSM2_log2.bw ")
    histone_list += (histone + "_E12_5 ") #The order of heatmap labels corresponding to the order of bw files.
    histone_list += (histone + "_CTL ")
    histone_list += (histone + "_SSM2 ")

In [ ]:
#Compute a heatmap matrix that combines all 16 different bw files in a specific order and based around select regions using bed files.
#The following will utilize non-normalized bw files.
#Two separate heatmaps will be ultimately generated, one with 3 different clusters using gene lists, and one that uses Hox genes.
#Will generate matrices that calculate scores 1 kb upstream and downstream of the transciption start site (TSS).

!computeMatrix reference-point \
    -S {bw_log2_list} \
    -R mm10_cluster1_genelist_NCBI_UCSC_filtered.bed mm10_cluster2_genelist_NCBI_UCSC_filtered.bed mm10_cluster3_genelist_NCBI_UCSC_filtered.bed -bs 25 -b 5000 -a 5000  \
    -out CTL_E12_5_SSM2_TSS_log2_5kb_clusters_filtered.matrix --missingDataAsZero --skipZeros \
    --samplesLabel {histone_list} \
    -p 25 --verbose

!computeMatrix reference-point \
    -S {bw_log2_list} \
    -R mm10_HoxGenes_genelist_NCBI_UCSC_filtered.bed -bs 25 -b 5000 -a 5000  \
    -out CTL_E12_5_SSM2_TSS_log2_5kb_HoxGenes_filtered.matrix --missingDataAsZero --skipZeros \
    --samplesLabel {histone_list} \
    -p 25 --verbose

In [ ]:
#Generate heatmaps in svg format for both types.

!plotHeatmap -m E12_5_CTL_SSM2_TSS_log2_5kb_clusters_filtered.matrix --colorMap RdBu --heatmapHeight 25 --heatmapWidth 5 \
    -out E12_5_CTL_SSM2_TSS_log2_5kb_clusters_filtered.svg --sortUsing max --legendLocation none \
    --regionsLabel "Cluster 1" "Cluster 2" "Cluster 3" --whatToShow "heatmap and colorbar" --outFileSortedRegions "E12_5_CTL_SSM2_TSS_5kb_log2_clusters.bed"

!plotHeatmap -m E12_5_CTL_SSM2_TSS_log2_5kb_HoxGenes_filtered.matrix --colorMap RdBu --heatmapHeight 10 --heatmapWidth 5 \
    -out E12_5_CTL_SSM2_TSS_log2_5kb_HoxGenes_filtered.svg --sortUsing max --legendLocation none \
    --regionsLabel "Hox Genes" --whatToShow "heatmap and colorbar" --outFileSortedRegions "E12_5_CTL_SSM2_TSS_5kb_log2_HoxGenes.bed"

# Program Versions Used

In [1]:
!pip freeze #Python packages and versions used in the conda environment

accessible-pygments==0.0.4
adjustText==0.8
aiohttp==3.9.3
aiohttp-cors==0.7.0
aiosignal==1.3.1
alabaster==0.7.13
anndata==0.9.2
annotated-types==0.6.0
annoy==1.17.3
anyio==4.2.0
appdirs==1.4.4
arboreto==0.1.6
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-lru==2.0.4
async-timeout==4.0.3
attr==0.3.2
attrs==23.2.0
Babel==2.14.0
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
backports.zoneinfo==0.2.1
bbknn==1.6.0
beautifulsoup4==4.12.3
bidict==0.22.1
bioservices==1.11.2
bleach==6.1.0
blessed==1.20.0
blosc2==2.0.0
bokeh==3.1.1
boltons==23.1.1
bs4==0.0.2
cachetools==5.3.2
cattrs==23.2.3
certifi==2024.2.2
cffi==1.16.0
charset-normalizer==3.3.2
click==8.1.7
cloudpickle==3.0.0
colorama==0.4.6
colorful==0.5.6
colorlog==6.8.2
comm==0.2.1
contourpy==1.1.1
cryptography==42.0.2
ctxcore==0.2.0
cycler==0.12.1
Cython==0.29.37
cytoolz==0.12.3
dask==2023.5.0
dataclasses-json==0.6.4
debugpy @ file

In [10]:
!pip show deeptools

Name: deepTools
Version: 3.5.4.post1
Summary: Useful tools for exploring deep sequencing data.
Home-page: 
Author: Fidel Ramirez, Devon P Ryan, Björn Grüning, Friederike Dündar, Sarah Diehl, Vivek Bhardwaj, Fabian Kilpert, Andreas S Richter, Steffen Heyne, Thomas Manke
Author-email: bioinfo-core@ie-freiburg.mpg.de
License: The file deeptools/cm.py is licensed under the BSD license, see a copy in that file. The remainder of the code is licensed under the MIT license:
        
        Copyright 2019 Max Planck Institute for Immunobiology and Epigenetics
        
        Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following cond

In [8]:
!./wigToBigWig

wigToBigWig v 2.9 - Convert ascii format wig file (in fixedStep, variableStep
or bedGraph format) to binary big wig format (bbi version: 4).
usage:
   wigToBigWig in.wig chrom.sizes out.bw
Where in.wig is in one of the ascii wiggle formats, but not including track lines
and chrom.sizes is a two-column file/URL: <chromosome name> <size in bases>
and out.bw is the output indexed big wig file.
If the assembly <db> is hosted by UCSC, chrom.sizes can be a URL like
  http://hgdownload.soe.ucsc.edu/goldenPath/<db>/bigZips/<db>.chrom.sizes
or you may use the script fetchChromSizes to download the chrom.sizes file.
If not hosted by UCSC, a chrom.sizes file can be generated by running
twoBitInfo on the assembly .2bit file.
options:
   -blockSize=N - Number of items to bundle in r-tree.  Default 256
   -itemsPerSlot=N - Number of data points bundled at lowest level. Default 1024
   -clip - If set just issue warning messages rather than dying if wig
                  file contains items off end of

In [3]:
!sort-bed --version

sort-bed
  citation: http://bioinformatics.oxfordjournals.org/content/28/14/1919.abstract
            https://doi.org/10.1093/bioinformatics/bts277
  version:  2.4.37 (typical)
  authors:  Scott Kuehn


In [6]:
!sort --version

sort (GNU coreutils) 8.30
Copyright (C) 2018 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.

Written by Mike Haertel and Paul Eggert.
